In [142]:
import pandas as pd
import os
import glob

filepath = "../../historical_data/BTCUSDT/BTCUSDT.parquet.gzip"
data = pd.read_parquet(filepath, engine='pyarrow')

In [143]:
import numpy as np

In [144]:
freq = 1
freq = "{}d".format(freq)
data_resampled = data.copy().resample(freq).last().dropna().iloc[:-1]

In [145]:
df = data_resampled.iloc[:31].copy()

In [146]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-01,13480.01,13480.01,13450.96,13452.00,3.685215
2018-01-02,14799.00,14799.97,14785.48,14799.96,26.488199
2018-01-03,14800.04,14800.04,14799.64,14799.91,6.267085
2018-01-04,14885.51,14940.00,14885.51,14939.39,17.975099
2018-01-05,16988.87,17047.92,16988.86,17021.36,20.574944


In [147]:
# return = (Pt - Pt-1) / Pt-1 = (Pt - Pt-1) - 1
df["returns"] = df.Close / df.Close.shift() - 1

In [148]:
df.dropna(inplace=True)

In [149]:
print(df.head())

                Open      High       Low     Close     Volume   returns
Date                                                                   
2018-01-02  14799.00  14799.97  14785.48  14799.96  26.488199  0.100205
2018-01-03  14800.04  14800.04  14799.64  14799.91   6.267085 -0.000003
2018-01-04  14885.51  14940.00  14885.51  14939.39  17.975099  0.009424
2018-01-05  16988.87  17047.92  16988.86  17021.36  20.574944  0.139361
2018-01-06  16967.50  17000.00  16960.02  16975.00   5.255252 -0.002724


In [150]:
df["ln_returns"] = np.log(1 + df.Close.pct_change())

In [151]:
df.dropna(inplace=True)

In [152]:
ema_s_val = 3
ema_l_val = 5

In [153]:
ema_s = df["Close"].ewm(span=ema_s_val-1, min_periods=ema_s_val).mean()
ema_l = df["Close"].ewm(span=ema_l_val-1, min_periods=ema_l_val).mean()

In [154]:
ema_s.dropna().head()

Date
2018-01-05    16370.024615
2018-01-06    16778.383000
2018-01-07    16555.878678
2018-01-08    15236.816813
2018-01-09    14581.919488
Freq: D, Name: Close, dtype: float64

In [155]:
ema_l.dropna().head()

Date
2018-01-07    16439.827453
2018-01-08    15659.489001
2018-01-09    15081.480872
2018-01-10    14844.915127
2018-01-11    14089.734597
Freq: D, Name: Close, dtype: float64

In [156]:
position = np.zeros(len(df))
position[ema_s > ema_l] = 1

In [157]:
position

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])

In [158]:
len(position)

29

In [159]:
import matplotlib.pyplot as plt

In [160]:
position = pd.Series(position, index=df.index).ffill().fillna(0)

In [161]:
len(position)

29

In [162]:
df = df.assign(position=position)

In [163]:
df.head()

,Open,High,Low,Close,Volume,returns,ln_returns,position
Date,,,,,,,,
2018-01-03,14800.04,14800.04,14799.64,14799.91,6.267085,-0.000003,-0.000003,0.0
2018-01-04,14885.51,14940.00,14885.51,14939.39,17.975099,0.009424,0.009380,0.0
2018-01-05,16988.87,17047.92,16988.86,17021.36,20.574944,0.139361,0.130468,0.0
2018-01-06,16967.50,17000.00,16960.02,16975.00,5.255252,-0.002724,-0.002727,0.0
2018-01-07,16435.00,16458.00,16435.00,16446.00,2.071036,-0.031163,-0.031659,1.0


In [164]:
ema_s_ser = pd.Series(ema_s, index=df.index).ffill().fillna(0)

In [165]:
ema_l_ser = pd.Series(ema_l, index=df.index).ffill().fillna(0)

In [166]:
df = df.assign(ema_s=ema_s_ser)

In [167]:
df = df.assign(ema_l=ema_l_ser)

In [168]:
df.head()

,Open,High,Low,Close,Volume,returns,ln_returns,position,ema_s,ema_l
Date,,,,,,,,,,
2018-01-03,14800.04,14800.04,14799.64,14799.91,6.267085,-0.000003,-0.000003,0.0,0.000000,0.000000
2018-01-04,14885.51,14940.00,14885.51,14939.39,17.975099,0.009424,0.009380,0.0,0.000000,0.000000
2018-01-05,16988.87,17047.92,16988.86,17021.36,20.574944,0.139361,0.130468,0.0,16370.024615,0.000000
2018-01-06,16967.50,17000.00,16960.02,16975.00,5.255252,-0.002724,-0.002727,0.0,16778.383000,0.000000
2018-01-07,16435.00,16458.00,16435.00,16446.00,2.071036,-0.031163,-0.031659,1.0,16555.878678,16439.827453


In [169]:
trades = df["position"].diff().abs().shift(1)

In [170]:
trades.head(10)

Date
2018-01-03    NaN
2018-01-04    NaN
2018-01-05    0.0
2018-01-06    0.0
2018-01-07    0.0
2018-01-08    1.0
2018-01-09    1.0
2018-01-10    0.0
2018-01-11    0.0
2018-01-12    0.0
Freq: D, Name: position, dtype: float64

In [171]:
strategy = df["position"].shift(1) * df["returns"]

In [172]:
strategy.head(10)

Date
2018-01-03         NaN
2018-01-04    0.000000
2018-01-05    0.000000
2018-01-06   -0.000000
2018-01-07   -0.000000
2018-01-08   -0.113462
2018-01-09   -0.000000
2018-01-10    0.000000
2018-01-11   -0.000000
2018-01-12    0.000000
Freq: D, dtype: float64

In [173]:
tc=0.00007
strategy_net = strategy - trades * tc

In [174]:
strategy_net.head(10)

Date
2018-01-03         NaN
2018-01-04         NaN
2018-01-05    0.000000
2018-01-06   -0.000000
2018-01-07   -0.000000
2018-01-08   -0.113532
2018-01-09   -0.000070
2018-01-10    0.000000
2018-01-11   -0.000000
2018-01-12    0.000000
Freq: D, dtype: float64

In [175]:
df = df.assign(strategy=strategy,
               trades=trades,
               strategy_net=strategy_net)

In [182]:
df.dropna(inplace=True)
df.head(10)

,Open,High,Low,Close,Volume,returns,ln_returns,position,ema_s,ema_l,strategy,trades,strategy_net
Date,,,,,,,,,,,,,
2018-01-05,16988.87,17047.92,16988.86,17021.36,20.574944,0.139361,0.130468,0.0,16370.024615,0.000000,0.000000,0.0,0.000000
2018-01-06,16967.50,17000.00,16960.02,16975.00,5.255252,-0.002724,-0.002727,0.0,16778.383000,0.000000,-0.000000,0.0,-0.000000
2018-01-07,16435.00,16458.00,16435.00,16446.00,2.071036,-0.031163,-0.031659,1.0,16555.878678,16439.827453,-0.000000,0.0,-0.000000
2018-01-08,14571.02,14607.02,14520.01,14580.00,30.949144,-0.113462,-0.120432,0.0,15236.816813,15659.489001,-0.113462,1.0,-0.113532
2018-01-09,14250.01,14277.45,14218.01,14254.92,15.806443,-0.022296,-0.022549,0.0,14581.919488,15081.480872,-0.000000,1.0,-0.000070
2018-01-10,14449.99,14500.00,14432.67,14500.00,16.100888,0.017193,0.017047,0.0,14527.298171,14844.915127,0.000000,0.0,0.000000
2018-01-11,12976.00,12999.98,12946.05,12975.99,19.033956,-0.105104,-0.111048,0.0,13493.040178,14089.734597,-0.000000,0.0,-0.000000
2018-01-12,13640.01,13647.40,13631.26,13646.99,0.721831,0.051711,0.050418,0.0,13595.675131,13911.559401,0.000000,0.0,0.000000
2018-01-13,14329.99,14341.40,14308.09,14329.99,6.311058,0.050048,0.048836,1.0,14085.221140,14079.541073,0.000000,0.0,0.000000


In [177]:
data = df.copy()

def calculate_performance():

    strategy_multiple_net = round(calculate_multiple(data.strategy_net), 3)
    strategy_multiple = round(calculate_multiple(data.strategy), 3)
    bh_multiple = round(calculate_multiple(data.returns), 3)
    outperf_net = round(strategy_multiple_net - bh_multiple, 3)
    outperf = round(strategy_multiple - bh_multiple, 3)
    num_of_trades = calculate_num_of_trades()
    num_samples = len(data)
    
    print("PERFORMANCE MEASURES:")
    print("\n")
    # print(f"Number of trades: {self.num_of_trades}")
    print("Multiple (Strategy):         {}".format(strategy_multiple))
    print("Multiple (Buy-and-Hold):     {}".format(bh_multiple))
    print(38 * "-")
    print("Out-/Underperformance:       {}".format(outperf))
    print(f"Number of Trades: {num_of_trades}")
    print(f"Strategy Multiple Net: {strategy_multiple_net}")
    

In [178]:
def calculate_num_of_trades():
    return int(data.trades.value_counts().get(1.0, 0))

In [179]:
calculate_performance()
# store strategy performance data for further plotting
#params_dic = {"freq": freq, "ema_s": ema_s_val, "ema_l": ema_l_val}
#self.dataploter.store_testcase_data(self.perf_obj, params_dic)  # comb[0] is current data freq

PERFORMANCE MEASURES:


Multiple (Strategy):         0.714
Multiple (Buy-and-Hold):     0.743
--------------------------------------
Out-/Underperformance:       -0.029
Number of Trades: 8
Strategy Multiple Net: 0.713


In [180]:
def calculate_multiple(series):
    return np.exp(series.sum())